In [ ]:
!apt update && apt upgrade
!cd exiftool && perl Makefile.PL && make && make test && make install
!exiftool -ver

In [ ]:
# !pip install arcticdb
# !pip install pandas
# !pip install pyexiftool
from arcticdb import Arctic, QueryBuilder
import exiftool
import sys
import os
import pandas as pd 
import numpy as np
import requests
import typing
import subprocess
import io
import json

In [ ]:
def _handle_errors(msg):
  print(Exception(msg))

In [ ]:
ac = Arctic(f"lmdb:///{os.getcwd()}/new_folder")
name_db = "testing.arctic"
try:
  ac.get_library(name_db)
  print("Found lib")
except:
  print("Not found")
try:
  ac.create_library(name_db)
  print("Created lib")
except:
  print("Lib exists")


ac.list_libraries()

# Populating dataframe with real and public data



In [ ]:
lib = ac[name_db]
try:
  lib.read("my_data")
  print("Data already existing, skipping")
except:
  try:
    df = pd.read_csv("https://dados.cvm.gov.br/dados/FI/DOC/EVENTUAL/DADOS/eventual_fi_2023.csv",sep=";",encoding = "unicode_escape")
    print("Created the dataframe with cvm data")
  except Exception as er:
    print(er)
    print("An error occurred when trying to generate the dataframe with the cvm data, modify or let a standard dataframe be generated as shown in the docs")
    NUM_COLUMNS=10
    NUM_ROWS=100_000
    df = pd.DataFrame(np.random.randint(0,100,size=(NUM_ROWS, NUM_COLUMNS)), columns=[f"COL_{i}" for i in range(NUM_COLUMNS)], index=pd.date_range('2000', periods=NUM_ROWS, freq='h'))
  lib.write("my_data", df)
# print(lib)
# data = lib.read("my_data",as_of=0)
# print(lib.tail("my_data",  5, as_of=0).data)

# data = lib.read("my_data")
# print(data)
# print(data.data)

# Create query builder using arcticdb





In [ ]:
def _download_files(link: str = "") -> typing.Union[bytes,None]:
    try:
        response = requests.get(link)
        response.raise_for_status()
        return response
    except requests.exceptions.RequestException as e:
        print(f"Ocorreu um erro ao baixar o arquivo {link}: {e}")
        return None

In [ ]:
def _run_exiftool_command_with_bytesio(content):
    command = ['exiftool', '-json', '-']
    process = subprocess.Popen(command, stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    output, error = process.communicate(input=content)
    return output, error

In [ ]:
query = QueryBuilder()
query = query[query["TP_DOC"] == "REGUL FDO"]
 
data = lib.read("my_data", query_builder=query).data[["CNPJ_FUNDO", "DT_RECEB", "TP_FUNDO", "LINK_ARQ"]]

for _index, _row in data.iterrows():
  print(_row["LINK_ARQ"])
  print(_row["DT_RECEB"])
  arq = _download_files(_row["LINK_ARQ"])
  try:
    arq_io = io.BytesIO(arq.content)
  except:
    print("An error occurred while trying to get the contents of the file")
    continue
  output, error = _run_exiftool_command_with_bytesio(arq_io.getvalue())

  # Verificar se ocorreu algum erro
  if error:
      print(f"Erro: {error.decode('utf-8')}")
  else:
      # Converter o output em um dicionário
      output_dict = json.loads(output.decode('utf-8'))
      print(output_dict[0])
      try:
        data.at[_index, "CreateDate"] = output_dict[0]["CreateDate"]
        data.at[_index, "ModifyDate"] = output_dict[0]["ModifyDate"]
      except:
        print("An error occurred while checking the parameters of CreateDate and ModifyDate")
  # break

data